In [2]:
#import pakages
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from bs4 import BeautifulSoup as bs
import string, time, re
from selenium.webdriver.common.keys import Keys
import csv

rate='' #creates an empty string for rate
name=[] #creates an empty list for names of customers
mydict={"Packname":[],"Names":[]} #creating a dictionary to store the data example: key:packname value:2019 Theme Nights - 5.2.19
execute_path = r'Drive\geckodriver' #path where gekodriver is installed
drive = webdriver.Firefox(executable_path=execute_path)
drive.implicitly_wait(5) # webdriver will implicitly wait for 5 seconds 
drive.get("https://app.hubspot.com/login") 
drive.implicitly_wait(5)

#Navigating to the email pack
username=drive.find_element_by_id(id_="username") #finds the element by its id
username.send_keys("info@nylizards.com") #input the username
password=drive.find_element_by_id(id_="password") 
password.send_keys("Lizards2015") #input the password
submit=drive.find_element_by_id(id_="loginBtn") 
submit.click()
Marketing=drive.find_element_by_id(id_="nav-primary-marketing-branch")
Marketing.click()
Email=drive.find_element_by_id(id_="nav-secondary-email")
Email.click()

f=open("outputff.csv",'w+') #opening a csv to write the data
count=1
i=0

#
while count<=19:	#no of pages
    count+=1 #increment count from 1
    while i<15:#no of packs per page
        
        check1=drive.find_elements_by_css_selector("tbody tr td.table-standard-column")[i] #Checks whether the email list is empty
        rate=check1.text #stores the number of percent (right end of the email) 
        #print('Hi')
        for item in rate:
            #print("after item")
            if rate=="0%": #if the rate is 0% it enters into the loop 
                i+=1
                print(i)
                print(rate)
                rate=rate.replace(rate,'') #empty rate variable
                    
            else: #if the rate is 0% then it won’t go into that email pack and run the following steps
                print(rate)
                a_elements = [] #created a new list 
                # waits for a sec to load the data in the page and finds the element
                content_blocks = WebDriverWait(drive, 1).until(\
                expected_conditions.presence_of_all_elements_located((By.CSS_SELECTOR, "tbody tr a[class='private-link uiLinkWithoutUnderline uiLinkDark']")))[i]
                #Getting the name of the email pack
                content= drive.find_elements_by_css_selector("tbody tr a[class='private-link uiLinkWithoutUnderline uiLinkDark']")[i].text
                print(content) #print the name of the email pack
                mydict["Packname"].append(content) #append it in the dictionary 
                i+=1
                print("pack no {p}".format(p=i)) #prints the no of page to keep a track of the program
                print("Page no {c}".format(c=count))
                
                x=content_blocks.get_attribute("href")# gets all the links of the email packs to individually                        open in different browser
                execute_path = r'Drive\geckodriver'
                drive = webdriver.Firefox(executable_path=execute_path)
                drive.get(x)
                time.sleep(5) #time for the page to load
                #enter login details
                username = WebDriverWait(drive, 5).until(\
                expected_conditions.presence_of_element_located((By.ID,"username")))
                #username=drive.find_element_by_id(id_="username")
                username.send_keys("info@nylizards.com")
                password=drive.find_element_by_id(id_="password")
                password.send_keys("Lizards2015")
                submit=drive.find_element_by_id(id_="loginBtn")
                submit.click()
                time.sleep(10)
                #find recipient and clicked 
                Recipients = WebDriverWait(drive, 7).until(\
                expected_conditions.presence_of_element_located((By.CSS_SELECTOR, "nav a[data-tab-id='RECIPIENTS']")))
                Recipients.click()
                Clicked = WebDriverWait(drive, 4).until(\
                expected_conditions.presence_of_element_located((By.CSS_SELECTOR, "ul[class='private-nav private-nav--level-0'] li:nth-of-type(4)")))
                Clicked.click()#if you want to navigate into any other option rather tahn clicked change the li:nth-of-type(x) where x would be your choice
                time.sleep(5)
                content=drive.find_element_by_css_selector("tbody")
                content1=content.find_elements_by_tag_name("tr a[class='private-link uiLinkWithoutUnderline uiLinkDark recipients-event-row-recipient-link']")
                y=0
                while y<=0: #This loop takes the names from the pack and write it into the csv file
                    y+=1        
                    for x in content1:
                        
                        names=x.text #takes the names of the customers
                                   
                        mydict["Names"].append(names)
                        f.write("\n"+mydict["Packname"][-1]+","+names)

                    print("Done!") #Print done as acknowledgement that we are done with that email pack
                    break
                    
                #writefile.close() 
                
                back_email=drive.find_element_by_css_selector("div.page i18n-string[data-key='ContentEmailUI.details.header.back']")
                back_email.click()# goes back to the page where there is the list of emails
                time.sleep(5)
            break
        
    drive.execute_script("window.scrollTo(0, document.body.scrollHeight);") #scrolls the page            
    next_page = WebDriverWait(drive, 3).until(\
    expected_conditions.presence_of_all_elements_located((By.CSS_SELECTOR, "nav.private-paginator span.private-paginator__numbers button")))[count]
    next_page.click()#takes you to the next page
    drive.execute_script("window.scrollTo(0, 0);") #scroll to the end of the page
    i=0 #resets the counter to 0 as we are going to the new page
f.close() #closes the csv file


WebDriverException: Message: 'geckodriver' executable needs to be in PATH. 
